In [ ]:
try:
    import tkFont
    import ttk
except ImportError:  # Python 3
    import tkinter.font as tkFont
    import tkinter.ttk as ttk
import pandas as pd
from IPython.display import display
from tkinter import *
from tkinter import messagebox
from tkinter.filedialog import askopenfilename
import pandas as pd
import random

tree = None
overlab = False
colorNum = 0

# 함수 모음
# 틀 갖추는 메소드
def setup_widgets():
    global tree
    #container = ttk.Frame()
    container=Frame(window)
    container.grid(row=15,column=0,sticky=W)
    #container.pack()
    # create a treeview with dual scrollbars
    tree = ttk.Treeview(columns=timeTable_header, show="headings")
    tree.bind("<Double-1>", OnDoubleClick)
    vsb = ttk.Scrollbar(orient="vertical",
        command=tree.yview)
    hsb = ttk.Scrollbar(orient="horizontal",
        command=tree.xview)
    tree.configure(yscrollcommand=vsb.set,
        xscrollcommand=hsb.set)
    tree.grid(column=0, row=0, sticky='nsew', in_=container)
    vsb.grid(column=1, row=0, sticky='ns', in_=container)
    hsb.grid(column=0, row=1, sticky='ew', in_=container)
    container.grid_columnconfigure(0, weight=1)
    container.grid_rowconfigure(0, weight=1)
    
# 트리 생성
def build_tree():
    global tree
    for col in timeTable_header:
        tree.heading(col, text=col.title(),
            command=lambda c=col: sortby(self.tree, c, 0))
        # adjust the column's width to the header string
        tree.column(col,
            width=tkFont.Font().measure(col.title()))
        
    for item in timeTable_list:
        tree.insert('', 'end', values=item)
        # adjust column's width if necessary to fit each value
        for ix, val in enumerate(item):
            col_w = tkFont.Font().measure(val)
            if tree.column(timeTable_header[ix],width=None)<col_w:
                tree.column(timeTable_header[ix], width=col_w)
        
def sortby(tree, col, descending):
    """sort tree contents when a column header is clicked on"""
    # grab values to sort
    data = [(tree.set(child, col), child) \
        for child in tree.get_children('')]
    # if the data to be sorted is numeric change to float
    #data =  change_numeric(data)
    # now sort the data in place
    data.sort(reverse=descending)
    for ix, item in enumerate(data):
        tree.move(item[1], '', ix)
    # switch the heading so it will sort in the opposite direction
    tree.heading(col, command=lambda col=col: sortby(tree, col, \
        int(not descending)))

# 트리뷰에서 더블클릭했을때 데이터 읽어오기
def OnDoubleClick(event):
    global tree
    global overlab
    global colorNum
    try:
        color = color_list[colorNum]
    except:
        messagebox.showinfo("Information","18학점 모두 선택하셨습니다.")
    
    item = tree.selection()[0]
    subjectTuple = tree.item(item,"values")
    #print(subjectTuple)
    #print(subjectTuple[4] +  subjectTuple[9])
    #print("you clicked on", tree.item(item,"values"))   
    
    df_excel = loadDataTable()
    n = df_excel[df_excel.강좌번호==subjectTuple[6]] 
    #print(n)
    
    if colorNum < 6:
        # dataframe에서 요일과 시간정보를 불러와 해당 시간표 작성하는 코드
        for o in n['요일'] + n['시간']:
            #print(o)
            #print("----------")
            d = str(o[0:1])
            #print(d)
            u = str(o[1:])
            #print(u)
            # 요일 위치
            dayNum = checkDay(d)
            s = str(u[0:2])
            e = str(u[6:8])
            gap = int(e) - int(s) + 1
            # 시간 위치
            timeNum = checkTime(s)
            # 시간표 작성
            writeTimeTable(timeNum, dayNum, subjectTuple[1], gap, color)
    
    if overlab:
        messagebox.showwarning("Warning","시간표가 이미 존재합니다.")
        overlab = False
    else:
        colorNum = colorNum + 1
    
def writeTimeTable(timeNum, dayNum, subject, gap, color):
    global overlab 
    for i in range(gap):
        temp = subjectEntry[timeNum + i][dayNum].get()
        if temp != "":
            overlab = True    
    if overlab:
        return
    else :
        for i in range(gap):
            subjectEntry[timeNum + i][dayNum].insert(END,subject)
            subjectEntry[timeNum + i][dayNum]["bg"] = color
                    
def checkDay(day):
    if(day == '월'):
        dayNum = 1
    elif(day == '화'):
        dayNum = 2
    elif(day == '수'):
        dayNum = 3
    elif(day == '목'):
        dayNum = 4
    elif(day == '금'):
        dayNum = 5
    else:
        dayNum = 0
    return dayNum

def checkTime(time):
    if (time == "09"):
        timeNum = 1
    elif (time == "10"):
        timeNum = 2
    elif (time == "11"):
        timeNum = 3
    elif (time == "12"):
        timeNum = 4
    elif (time == "13"):
        timeNum = 5
    elif (time == "14"):
        timeNum = 6
    elif (time == "15"):
        timeNum = 7
    elif (time == "16"):
        timeNum = 8
    elif (time == "17"):
        timeNum = 9
    elif (time == "18"):
        timeNum = 10
    elif (time == "19"):
        timeNum = 11
    else: 
        timeNum = 12   
    return timeNum

def donothing():
    filewin= Toplevel(timetableFrame) # 새로운창띄우기
    Button(filewin, text="Do nothing button").pack()
    
def openFile():
    global fname
    fname= askopenfilename(filetypes = (("Excel File" , "*.xlsx"),("All Files","*.*")))
    messagebox.showinfo('You choosed:', fname)
    #loadData()
    genDataTree()
    
def loadData():
    Location='timetable.xlsx'
    df_excel=pd.read_excel(Location,0,header=0,converters={'강좌번호':str})
    df_excel=df_excel.fillna(method='ffill')
    df_excel['강의실']=df_excel['시간(강의실)'].apply(lambda x:x[len(x)-5:len(x)-1])
    df_excel['요일']=df_excel['시간(강의실)'].apply(lambda x: x[0:1])
    df_excel['시간']=df_excel['시간(강의실)'].apply(lambda x: x[1:len(x)-6])
    df_excel=df_excel.drop(['비  고','시간(강의실)'],axis=1)
    #display(df_excel)
        
# the test data ...
def loadDataTable():
    Location='timetable.xlsx'
    df_excel=pd.read_excel(Location,0,header=0,converters={'강좌번호':str})
    df_excel=df_excel.fillna(method='ffill')
    df_excel['강의실']=df_excel['시간(강의실)'].apply(lambda x:x[len(x)-5:len(x)-1])
    df_excel['요일']=df_excel['시간(강의실)'].apply(lambda x: x[0:1])
    df_excel['시간']=df_excel['시간(강의실)'].apply(lambda x: x[1:len(x)-6])
    df_excel=df_excel.drop(['비  고','시간(강의실)'],axis=1)
    #display(df_excel)
    return df_excel

def genDataTree():
    setup_widgets()
    build_tree()

color_list = ["#ffa500", "#ffc0cb", "#6495ed", "#ff4500", "#f08080", "#adff2f"]

timeTable_header = ['학년', '교과목명', '한글코드', '학점', '시간', '담당교수', '강좌번호',
                    '제한인원', '강의실', '요일']      
reader = loadDataTable()

grade = reader['학년']
name = reader['교과목명']
code = reader['한글코드']
score = reader['학점']
time = reader['시간']
professor = reader['담당교수']
classNum = reader['강좌번호']
personNum = reader['제한인원']
place = reader['강의실']
day = reader['요일']

timeTable_list = zip(grade, name, code, score, time, professor, classNum, personNum, place, day)

#-------------------------------------------------------------------------------------------------------

# 윈도우창 열기
window = Tk()
window.title("My Project")

# Frame
timetableFrame=Frame(window)
timetableFrame.grid(row=0,column=0,sticky=W)
# Frame
gradeFrame=Frame(window)
gradeFrame.grid(row=16,column=0,sticky=W)

# Menu 생성하기
menubar= Menu(timetableFrame)
filemenu= Menu(menubar, tearoff=0)
filemenu.add_command(label="New", command=donothing)
filemenu.add_command(label="Open", command=openFile)
filemenu.add_separator()
filemenu.add_command(label="Exit", command=window.quit)
menubar.add_cascade(label="File", menu=filemenu)
window.config(menu=menubar)

# Label 생성하기
Label(timetableFrame, text="산업경영공학과시간표").grid(row=2, column=0, columnspan=2)

# Entry table 생성하기
height = 12
width = 6
dayList = ['월', '화', '수', '목', '금']  # 요일 리스트
timeList = ['09:00 ~ 09:50', '10:00 ~ 10:50', '11:00 ~ 11:50', '12:00 ~ 12:50',
            '13:00 ~ 13:50', '14:00 ~ 14:50', '15:00 ~ 15:50', '16:00 ~ 16:50',
            '17:00 ~ 17:50', '18:00 ~ 18:50', '19:00 ~ 19:50']

# 시간 요일 라벨 생성
for i in range(height): #Rows
    for j in range(width): #Columns
        if i == 0 :
            if j != 0:
                dayName = dayList[j-1]
                b = Label(timetableFrame, text=dayName)
                b.grid(row=i+3, column=j)
        else :
            if j == 0:
                time = timeList[i-1]
                b = Label(timetableFrame, text=time)
                b.grid(row=i+3, column=j)
                
# 과목 엔트리 리스트 생성
subjectEntry = [[0]*width for i in range(height)]

for i in range(height): #Rows
    for j in range(width): #Columns
        if i != 0:
            if j != 0:
                subjectEntry[i][j] = Entry(timetableFrame, text="", justify="center")
                subjectEntry[i][j].grid(row=i+3,column=j)
                
# Option 선택하기
control_variable = StringVar(timetableFrame)
OPTION_TUPLE = ("Option 1", "Option 2", "Option 3")
optionmenu_widget= OptionMenu(window, control_variable, *OPTION_TUPLE)
optionmenu_widget.grid(row=13, column=0)

window.mainloop()